In [21]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/bacterial-darmatosis/bacterial_dermatosis.jpeg
/kaggle/input/hypersensitivity-d/hypersensitivity_dermatitis.jpg
/kaggle/input/hypersensitivity-d/hypersensitivity_dermatitis_1
/kaggle/input/dogs-skin-disease-dataset/Dogs/Bacterial_dermatosis/vx.jpg
/kaggle/input/dogs-skin-disease-dataset/Dogs/Bacterial_dermatosis/56.jpg
/kaggle/input/dogs-skin-disease-dataset/Dogs/Bacterial_dermatosis/89.jpg
/kaggle/input/dogs-skin-disease-dataset/Dogs/Bacterial_dermatosis/pic456.jpg
/kaggle/input/dogs-skin-disease-dataset/Dogs/Bacterial_dermatosis/zdfg.jpg
/kaggle/input/dogs-skin-disease-dataset/Dogs/Bacterial_dermatosis/bac4_png.rf.d5c6154580356c25885593026f7372f4.jpg
/kaggle/input/dogs-skin-disease-dataset/Dogs/Bacterial_dermatosis/bacterial_Image_1_jpg.rf.f5236c4b5b2eeb1383ed9667d012d8b2.jpg
/kaggle/input/dogs-skin-disease-dataset/Dogs/Bacterial_dermatosis/bacc1_jpg.rf.0c5595790601e8855cab203843db6628.jpg
/kaggle/input/dogs-skin-disease-dataset/Dogs/Bacterial_dermatosis/bacterial-infec

In [22]:
# ✅ Dog Skin Disease Detection with Medicine Recommendation

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
from sklearn.utils.multiclass import unique_labels
import os

In [23]:
# ✅ Paths (use correct dataset path)
train_dir = '/kaggle/input/dogs-skin-disease-dataset/Dogs'
IMG_SIZE = (224, 224)
BATCH = 32


In [24]:
# ✅ Data generators
train_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    zoom_range=0.3,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

train_data = train_gen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH,
    class_mode='categorical',
    subset='training'
)

val_data = train_gen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH,
    class_mode='categorical',
    subset='validation'
)

Found 354 images belonging to 4 classes.
Found 85 images belonging to 4 classes.


In [25]:
# ✅ Model building
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
outputs = Dense(train_data.num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=outputs)


In [26]:
# ✅ Freeze base model and compile
for layer in base_model.layers:
    layer.trainable = False
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_data, validation_data=val_data, epochs=15)


Epoch 1/15
12/12 ━━━━━━━━━━━━━━━━━━━━ 23s 1s/step - accuracy: 0.3120 - loss: 1.5889 - val_accuracy: 0.3059 - val_loss: 1.4272
Epoch 2/15
12/12 ━━━━━━━━━━━━━━━━━━━━ 9s 762ms/step - accuracy: 0.4396 - loss: 1.2660 - val_accuracy: 0.3529 - val_loss: 1.3799
Epoch 3/15
12/12 ━━━━━━━━━━━━━━━━━━━━ 9s 786ms/step - accuracy: 0.4697 - loss: 1.1723 - val_accuracy: 0.3294 - val_loss: 1.4786
Epoch 4/15
12/12 ━━━━━━━━━━━━━━━━━━━━ 9s 726ms/step - accuracy: 0.5323 - loss: 1.1015 - val_accuracy: 0.3647 - val_loss: 1.3873
Epoch 5/15
12/12 ━━━━━━━━━━━━━━━━━━━━ 9s 734ms/step - accuracy: 0.5852 - loss: 1.0224 - val_accuracy: 0.4118 - val_loss: 1.2838
Epoch 6/15
12/12 ━━━━━━━━━━━━━━━━━━━━ 9s 750ms/step - accuracy: 0.5656 - loss: 1.0011 - val_accuracy: 0.3176 - val_loss: 1.4087
Epoch 7/15
12/12 ━━━━━━━━━━━━━━━━━━━━ 9s 765ms/step - accuracy: 0.6191 - loss: 0.9395 - val_accuracy: 0.3647 - val_loss: 1.3403
Epoch 8/15
12/12 ━━━━━━━━━━━━━━━━━━━━ 9s 737ms/step - accuracy: 0.6000 - loss: 0.9298 - val_accuracy: 0.40

In [27]:
# ✅ Fine-tune last layers
for layer in base_model.layers[-40:]:
    layer.trainable = True
model.compile(optimizer=tf.keras.optimizers.Adam(1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_data, validation_data=val_data, epochs=15)


Epoch 1/15
12/12 ━━━━━━━━━━━━━━━━━━━━ 39s 2s/step - accuracy: 0.4410 - loss: 1.3616 - val_accuracy: 0.4235 - val_loss: 1.4042
Epoch 2/15
12/12 ━━━━━━━━━━━━━━━━━━━━ 9s 725ms/step - accuracy: 0.4949 - loss: 1.2405 - val_accuracy: 0.4000 - val_loss: 1.4331
Epoch 3/15
12/12 ━━━━━━━━━━━━━━━━━━━━ 9s 762ms/step - accuracy: 0.5054 - loss: 1.2282 - val_accuracy: 0.4000 - val_loss: 1.4801
Epoch 4/15
12/12 ━━━━━━━━━━━━━━━━━━━━ 9s 731ms/step - accuracy: 0.5131 - loss: 1.2061 - val_accuracy: 0.4824 - val_loss: 1.3818
Epoch 5/15
12/12 ━━━━━━━━━━━━━━━━━━━━ 9s 746ms/step - accuracy: 0.5260 - loss: 1.0802 - val_accuracy: 0.4353 - val_loss: 1.3723
Epoch 6/15
12/12 ━━━━━━━━━━━━━━━━━━━━ 9s 727ms/step - accuracy: 0.5684 - loss: 1.0171 - val_accuracy: 0.3765 - val_loss: 1.4805
Epoch 7/15
12/12 ━━━━━━━━━━━━━━━━━━━━ 9s 754ms/step - accuracy: 0.5792 - loss: 0.9992 - val_accuracy: 0.4471 - val_loss: 1.5043
Epoch 8/15
12/12 ━━━━━━━━━━━━━━━━━━━━ 9s 737ms/step - accuracy: 0.6174 - loss: 0.9914 - val_accuracy: 0.49

In [28]:
# ✅ Save model
model.save("dog_skin_disease_model.h5")


In [29]:
# ✅ Evaluate with classification report
y_pred = model.predict(val_data)
y_true = val_data.classes
y_pred_classes = np.argmax(y_pred, axis=1)
class_labels = list(val_data.class_indices.keys())
labels_present = unique_labels(y_true, y_pred_classes)
label_names_present = [class_labels[i] for i in labels_present]

print("\nConfusion Matrix:")
print(confusion_matrix(y_true, y_pred_classes))
print("\nClassification Report:")
print(classification_report(y_true, y_pred_classes, target_names=label_names_present))


3/3 ━━━━━━━━━━━━━━━━━━━━ 7s 2s/step

Confusion Matrix:
[[ 1  3 11  3]
 [ 0  8  9 10]
 [ 0  5 14  4]
 [ 1  7  5  4]]

Classification Report:
                                      precision    recall  f1-score   support

                Bacterial_dermatosis       0.50      0.06      0.10        18
                   Fungal_infections       0.35      0.30      0.32        27
                             Healthy       0.36      0.61      0.45        23
Hypersensitivity_allergic_dermatosis       0.19      0.24      0.21        17

                            accuracy                           0.32        85
                           macro avg       0.35      0.30      0.27        85
                        weighted avg       0.35      0.32      0.29        85



In [30]:
# ✅ Prediction function with manual class indices
class_indices = {
    'bacterial_dermatosis': 0,
    'fungal_infection': 1,
    'healthy': 2,
    'hypersensitivity_dermatitis': 3
}
inv_map = {v: k for k, v in class_indices.items()}

def predict_image(img_path, model):
    img = image.load_img(img_path, target_size=IMG_SIZE)
    arr = image.img_to_array(img) / 255.0
    pred = model.predict(arr[np.newaxis, ...])[0]
    idx = np.argmax(pred)
    confidence = pred[idx]
    label = inv_map[idx]
    return label, confidence

In [31]:
# ✅ Predict uploaded test image
img_path = "/kaggle/input/dogs-skin-disease-dataset/Dogs/Bacterial_dermatosis/Bacterial_2_jpg.rf.57897be3c7eacc71cc9ef501002520e2.jpg"  # Replace this path
label, confidence = predict_image(img_path, model)
print(f"Predicted label: {label} (Confidence: {confidence:.2f})")


1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
Predicted label: bacterial_dermatosis (Confidence: 0.90)


In [32]:
# ✅ Treatment suggestion
treatments = {
    'bacterial_dermatosis': [
    'Chlorhexidine 4% shampoo (antibacterial wash)','Mupirocin ointment (topical antibiotic)',
    'Cephalexin (oral antibiotic, 22–30 mg/kg twice daily)','Clindamycin (oral, 5.5 mg/kg twice daily)',
    'Amoxicillin-clavulanate (Clavamox, oral)','Enrofloxacin (Baytril) for resistant infections (use with caution)',
    'Medicated wipes (e.g., chlorhexidine or benzoyl peroxide)'
],
    'fungal_infection': [
    'Miconazole shampoo or lotion (topical)','Ketoconazole (oral, 5–10 mg/kg once daily)',
    'Itraconazole (oral, 5 mg/kg once daily)','Lime sulfur dips (for ringworm)',
    'Chlorhexidine + Miconazole shampoo','Clotrimazole cream (for localized spots)',
    'Terbinafine (oral antifungal)'
],
    'hypersensitivity_dermatitis': [
    'Hydrocortisone 1% cream (topical steroid)','Prednisone (oral steroid, 0.5–1 mg/kg/day short-term)',
    'Apoquel (Oclacitinib, 0.4–0.6 mg/kg twice daily)','Cytopoint (lokivetmab, monoclonal antibody injection)',
    'Diphenhydramine (Benadryl, 2–4 mg/kg up to 3x daily)','Omega-3 fatty acid supplements (anti-inflammatory)',
    'Medicated anti-itch sprays (chlorhexidine or pramoxine)','Hypoallergenic or novel protein diet'
],
    'healthy': ['No treatment needed – maintain regular grooming and diet']
}

if label in treatments:
    print("💊 Suggested medicines:", treatments[label])
else:
    print(f"⚠️ No treatment info found for: {label}")

if confidence < 0.5:
    print("⚠️ Low confidence. Consider reviewing the image manually.")


💊 Suggested medicines: ['Chlorhexidine 4% shampoo (antibacterial wash)', 'Mupirocin ointment (topical antibiotic)', 'Cephalexin (oral antibiotic, 22–30 mg/kg twice daily)', 'Clindamycin (oral, 5.5 mg/kg twice daily)', 'Amoxicillin-clavulanate (Clavamox, oral)', 'Enrofloxacin (Baytril) for resistant infections (use with caution)', 'Medicated wipes (e.g., chlorhexidine or benzoyl peroxide)']
